In [ ]:
code = """

#include <bits/stdc++.h>
using namespace std;

void add(vector<int> &a,vector<int> &b,vector<int> &c,int size){
  for(int i = 0;i<size;i++){
    c[i] = a[i] + b[i];
  }

}
__global__ void addgpu(int *a,int *b,int *c,int size){
  int tid = blockIdx.x*blockDim.x + threadIdx.x;
  if(tid < size){
    c[tid] = a[tid] + b[tid];
  }
}
void print(vector<int> &a){
  for(auto it : a){
    cout<<it<<" ";
  }
  cout<<endl;
}

int main(){

  int n;
  cin>>n;
  vector<int> a(n),b(n),c(n);
  for(int i = 0;i<n;i++){
    cin>>a[i];
  }
  for(int i = 0;i<n;i++){
    cin>>b[i];
  }
  add(a,b,c,n);
  print(a);
  print(b);
  print(c);

  int gpusize = n*sizeof(int);
  int *da,*db,*dc;
  //memory allocate
  cudaMalloc(&da,gpusize);
  cudaMalloc(&db,gpusize);
  cudaMalloc(&dc,gpusize);

  //copy data
  cudaMemcpy(da,a.data(),gpusize,cudaMemcpyHostToDevice);
  cudaMemcpy(db,b.data(),gpusize,cudaMemcpyHostToDevice);

  int Threadperblock = 256;
  int blockpergrid = (Threadperblock + n - 1)/Threadperblock;

  addgpu<<<blockpergrid,Threadperblock>>>(da,db,dc,n);
  cudaGetLastError();
  cudaDeviceSynchronize();
  cout<<endl;
  cudaMemcpy(c.data(),dc,gpusize,cudaMemcpyDeviceToHost);
  cout<<"array a: ";
  print(a);
  cout<<"array b: ";
  print(b);
  cout<<"array c: ";
  print(c);
  cudaFree(da);
  cudaFree(db);
  cudaFree(dc);
  return 0;



}
"""

with open("add.cu","w") as f:
  f.write(code)
!nvcc -arch=sm_75 add.cu -o add
!./add

5
1
2
3
4
5
1
2
3
4
5
1 2 3 4 5 
1 2 3 4 5 
2 4 6 8 10 

array a: 1 2 3 4 5 
array b: 1 2 3 4 5 
array c: 2 4 6 8 10 
